# Scraping Twitter Using snscrape
<br>Package Github: https://github.com/JustAnotherArchivist/snscrape
<br>This notebook will be using the development version of snscrape

Article Read-Along: https://medium.com/better-programming/how-to-scrape-tweets-with-snscrape-90124ed006af

### Author: Martin Beck

<b>Dependencies: </b> 
- Your <b>Python</b> version must be <b>3.8</b> or higher. The development version of snscrape will not work with Python 3.7 or lower. You can download the latest Python version [here](https://www.python.org/downloads/).
- <b>Development version of snscrape</b>, uncomment the pip install line in the below cell to pip install in the notebook if you don't already have it.
- <b>Pandas</b>, the dataframes allows easy manipulation and indexing of data, this is more of a preference but is what I follow in this notebook.

In [1]:
#!pip install git+https://github.com/JustAnotherArchivist/snscrape.git

# Imports
import os
import pandas as pd
import itertools
import snscrape.modules.twitter as sntwitter

# Consulta por búsqueda de texto
El siguiente código buscará tweets a través de los siguientes términos:

* 'Dolor vacuna'
* 'efecto vacuna', 'efecto astrazeneca', etc.
* 'reacción vacuna'
* 'vacuna #AstraZeneca', 'vacuna #Sputnik', 'vacuna #j&j', etc.
* sputnik v, moderna
* 'vacuna sintoma'
* etc.

La búsqueda será de los años 2020, 2021 y 2022, y que serán almacenados en un archivo CSV

In [19]:
latitud = '19.309917325231165'   
longitud = '-99.12243737329997'
#radio de distancia a partir de 4
radio = '31.28km' 

localizacion = latitud + ',' + longitud + ',' + radio

#tweet_count = 100

#df_coord = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    #'efecto vacuna since:2022-01-01 until:2022-01-31 geocode:"{}"'.format(localizacion)).get_items(),tweet_count))[['id','url','date','content','likeCount','retweetCount']]

df_coord = pd.DataFrame((sntwitter.TwitterSearchScraper(
    'efecto sputnik since:2020-12-24 until:2022-06-17 geocode:"{}"'.format(localizacion)).get_items()))[['id','url','date','content','likeCount','retweetCount','replyCount']]

df_coord['latitud'] = latitud
df_coord['longitud'] = longitud
df_coord['radio'] = radio

df_coord.to_csv('efectos_covid.csv', sep=',', index=False,mode="w+")

# Pre procesamiento

In [1]:
import re
#!pip install emoji
import emoji
import pandas as pd
import numpy as np
#!pip install unidecode
from unidecode import unidecode

import nltk 
from nltk.tokenize import TweetTokenizer
#nltk.download('stopwords')
from nltk.corpus import stopwords

#!pip install pyspellchecker
from spellchecker import SpellChecker

from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
#nltk.download('omw')
from nltk.corpus import wordnet

#!pip install spacy
import spacy
#!python -m spacy download es
nlp = spacy.load('es_core_news_sm')

In [56]:
df_tw = pd.read_csv("primera_dosis.csv")
df_tw.head()

,id,url,date,content,likeCount,retweetCount,replyCount,latitud,longitud,radio,tweet_tokenizado
0,1541883888175292419,https://twitter.com/mugroson/status/1541883888...,2022-06-28 20:39:00+00:00,Y los niños que tres años tarde reciben si pri...,0,0,0,19.309917,-99.122437,31.28km,"['nino', 'tres', 'ano', 'tarde', 'recibir', 's..."
1,1541453091286810625,https://twitter.com/TROLELLO/status/1541453091...,2022-06-27 16:07:10+00:00,@GobCDMX Y se vuelven a equivocar y ponen cita...,1,0,0,19.309917,-99.122437,31.28km,"['volver', 'equivocar', 'poner', 'cita', 'segu..."
2,1541096949314723841,https://twitter.com/keniajh2/status/1541096949...,2022-06-26 16:31:59+00:00,https://t.co/c6lTOzU5ph\nhttps://t.co/uLl8OU9u...,0,0,1,19.309917,-99.122437,31.28km,"['tambien', 'podran', 'vacunar', 'adulto', 'bi..."
3,1539051546922582017,https://twitter.com/Marlem_mst/status/15390515...,2022-06-21 01:04:17+00:00,"Pues para tener tres dosis, el dolor de articu...",0,0,0,19.309917,-99.122437,31.28km,"['pues', 'tener', 'tres', 'dosis', 'dolor', 'a..."
4,1538962237720866817,https://twitter.com/damcastle/status/153896223...,2022-06-20 19:09:24+00:00,#VACUNACIONCOVID19 se estarán vacunando en el ...,2,2,0,19.309917,-99.122437,31.28km,"['estar', 'vacunar', 'emiliano', 'zapata', 'pe..."


In [24]:
def remover_links(tweet):
    tweet = re.sub(r'http\S+', '', tweet)   # quitar http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # quitar bitly links
    tweet = tweet.strip('[link]')   # quitar [links]
    
    return tweet

def remover_menciones(tweet):
    tweet = re.sub(r"@\S+", "", tweet)
    
    return tweet

def rep(m):
    s = m.group(1)
    return ' '.join(re.split(r'(?=[A-Z])', s))

def separar_palabras_hashtag(tweet):
    tweet = re.sub(r'#(\w+)', rep, tweet)
    
    return tweet

def separar_numeros_de_letras(tweet):
    tweet = re.findall(r'\D+|\d+', tweet)
    
    return lista_a_cadena(tweet)
    
def remover_hashtag(tweet):  
    tweet = re.sub(r'#', '', tweet)
    
    return tweet

def arreglar_abreviaciones(tweet):
    palabras = tweet.split()

    abreviaciones = {'d': 'de',
               'x': 'por',
               'xa': 'para',
               'as': 'has',
               'q': 'que',
               'k': 'que',
               'dl': 'del',
               'xq': 'porqué',
               'dr': 'doctor',
               'dra': 'doctora',
               'sr': 'señor',
               'sra': 'señora',
                'srta':'señorita',
               'm': 'me',
               'aprox': 'aproximadamente',
               'hrs': 'horas',
                '1er': 'primer',
                '1era': 'primera',
                '1ra' : 'primera',
                '1ra': 'primera',
                '2do': 'segundo',
                '2da': 'segunda',
                '3er': 'tercer',
                '3ero': 'tercero',
                '3era': 'tercera',
                '3ra' : 'tercera',
                'jj': 'johnson',
                'amixs': 'amigos',
                'pos': 'pues',
                'ok': 'está bien',
                'lit': 'literal',
                'vs': 'contra',
                'ptm': 'puta madre',
                'alch': 'al chile',
                'desminulleron' : 'disminuyeron',
                'días': 'día',
                'kg': 'kilogramo',
                'mg': 'miligramo',
                'echo' : 'hecho',
                'mins' : 'minutos',
                'min' : 'minuto',
                'pa': 'para',
                'holi' : 'hola'}
    
        
    tweet = " ".join([abreviaciones[palabra] if palabra in abreviaciones.keys() else palabra for palabra in palabras])

    return tweet

def arreglar_nombres_vacunas(tweet):
    i = 0
    
    palabras = tweet.split()
    
    tamañio = len(palabras)
    
    for palabra in palabras:
        #normaliza el nombre de la vacuna inglesa
        if palabra == 'astra':
            palabras[i] = 'astrazeneca'
            #print(tamañio)
            #print(i)
            #print(palabras)
            if not (i + 1 == tamañio):
                #print(i+1)
                #print(palabras)
                if palabras[i + 1] == 'zeneca':
                    tamañio = tamañio - 1
                    del palabras[i + 1]
                    
                elif palabras[i + 1] == 'seneca':
                    tamañio = tamañio - 1
                    del palabras[i + 1]
                    
        elif palabra == 'aztra':
            palabras[i] = 'astrazeneca'
            
            if not (i + 1 == tamañio):
                if palabras[i + 1] == 'zeneca':
                    tamañio = tamañio - 1
                    del palabras[i + 1]
                elif palabras[i + 1] == 'seneca':
                    tamañio = tamañio - 1
                    del palabras[i + 1]
                    
        elif palabra == 'zeneca':
            palabras[i] = 'astrazeneca'
        
        elif palabra == 'aztrazeneca':
            palabras[i] = 'astrazeneca'
            
        elif palabra == 'astrazeneca':
            palabras[i] = 'astrazeneca'
            
        elif palabra == 'aztraceneca':
            palabras[i] = 'astrazeneca'
            
        elif palabra == 'astraseneca':
            palabras[i] = 'astrazeneca'
            
        i+=1
    
    return lista_a_cadena(palabras)
    
def remover_estilos(tweet):  
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    
    return tweet

def remover_numeros(tweet):
    tweet = re.sub(r'[0-9]', '', tweet)
    
    return tweet

def minusculas(tweet):
    tweet = tweet.lower()
    
    return tweet

def remover_puntuaciones(tweet):
    puntuaciones = '!"$%&\'()*+,-./:;<=>¿?[\\]^_`{|}~•@º'
    tweet = re.sub('[' + puntuaciones + ']+', ' ', tweet) 
    
    #Remover comillas
    tweet = tweet.replace("’", " ")
    tweet = tweet.replace("“", '')
    tweet = tweet.replace("”", '')
    
    #Remover puntos de suspensión
    tweet = tweet.replace('\u2026', '')
    
    return tweet

def remover_espacios(tweet):
    tweet = re.sub('\s+', ' ', tweet)
    
    return tweet

def remover_emojis(tweet):
    tweet = emoji.replace_emoji(tweet, replace='')
    #tweet = re.sub(emoji.get_emoji_regexp(), r"", tweet)
    
    return tweet

def remover_letras_repetidas(tweet):
    tweet = re.sub(r'(.)\1{2,}',r'\1', tweet)
    
    return tweet

def remover_signos_diacríticos(tweet):
    #Conversión de la letra ñ a n
    tweet = re.sub(u"[ñ]", 'n', tweet)
    
    #Remover signos diacríticos
    tweet = unidecode(tweet)
    
    return tweet

def tokenizar(tweet):
    tk = TweetTokenizer()
    tweet_tokenizado = tk.tokenize(tweet) 
    
    return tweet_tokenizado

def remover_stopwords(tweet_tokenizado):   
    stop_words = stopwords.words('spanish')
    tweet_tokenizado = [palabra for palabra in tweet_tokenizado if palabra not in stop_words]
    
    return tweet_tokenizado

def remover_único_token(tweet_tokenizado):
    a=list(tweet_tokenizado)
    
    for palabra in a:
        if len(palabra) == 1 :
            tweet_tokenizado.remove(palabra)
            
    return tweet_tokenizado

def corrector_otográfico(tweet):
    spell = SpellChecker(language='es')
    
    #Agrega nuevos términos al corrector ortográfico
    spell.word_frequency.load_text_file('terminos.txt')

    spell.word_frequency.load_words(['covid','sputnik', 'astrazeneca','pfizer','biontech','pfizerbiontech',
                                    'cansino','moderna','janssen','johnson','booster','imss','gatell','sars',
                                     'cov','gripita', 'sarscov'])
    
    palabras = tweet.split()
    
    k = 0
    
    for palabra in palabras:
        palabra_correcta = spell.correction(palabra)
        palabras[k] = palabra_correcta
        
        k+=1 
        
    return palabras

def lista_a_cadena(s):
    str1 = " "
   
    return (str1.join(s))

def lematizar(tweet_tokenizado):
    lista = []
    
    doc = nlp(tweet_tokenizado)
    
    for token in doc:
        if str(token) == 'astrazeneca':
            lista.append('astrazeneca')
        elif str(token) == 'gatell':
            lista.append('gatell') 
        elif str(token) == 'imss':
            lista.append('imss')
        elif str(token) == 'sars':
            lista.append('sars')
        elif str(token) == 'cov2':
            lista.append('cov2')
        elif str(token) == 'meme':
            lista.append('meme')
        elif str(token) == 'memes':
            lista.append('memes')
        elif str(token) == 'manana':
            lista.append('manana')
        elif str(token) == 'tos':
            lista.append('tos')
        else:
            lema = token.lemma_.lower()
            lema = remover_signos_diacríticos(lema)
            
            if re.search(r"\s", lema):
                aux = lema.split(" ")
                lema = aux[0]
                
            lista.append(lema)

    return lista       
                
    #lemas = [tok.lemma_.lower() for tok in doc]    
    #return lemas

In [57]:
content = df_tw['content']
lista_tokens = []

for i in range(len(content)):
    tweet = content[i]
    tweet = remover_links(tweet)
    tweet = remover_menciones(tweet)
    tweet = remover_estilos(tweet)
    tweet = separar_palabras_hashtag(tweet)
    tweet = minusculas(tweet)
    tweet = remover_puntuaciones(tweet)
    tweet = remover_espacios(tweet)
    tweet = remover_emojis(tweet)
    tweet = remover_letras_repetidas(tweet)
    tweet = arreglar_abreviaciones(tweet)
    tweet = separar_numeros_de_letras(tweet)
    tweet = remover_numeros(tweet)
    tweet = arreglar_abreviaciones(tweet)
    tweet = arreglar_nombres_vacunas(tweet) 
    tweet_tokenizado = tokenizar(tweet)
    tweet_tokenizado = remover_stopwords(tweet_tokenizado)
    tweet_tokenizado = lista_a_cadena(tweet_tokenizado)
    tweet_tokenizado = remover_signos_diacríticos(tweet_tokenizado)
    tweet_tokenizado = corrector_otográfico(tweet_tokenizado)
    tweet_tokenizado = lista_a_cadena(tweet_tokenizado)
    tweet_tokenizado = lematizar(tweet_tokenizado) 
    tweet_tokenizado = remover_único_token(tweet_tokenizado) 
    
    lista_tokens.append(tweet_tokenizado)

In [58]:
df_tw['tweet_tokenizado'] = lista_tokens

df_tw.to_csv('primera_dosis.csv', sep=',', index=False,mode="w+")